In [56]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fracture-multi-region-x-ray-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4854718%2F8201044%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240830%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240830T043158Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D58f12c63bebc33f5d002f1eec3fd7d213250b69483ad3704481a486506f75b9dfe0e20516700c702fbcb5178f2b11456d9974c71aa746a49ed218e852fd29f8db7547deb4ec38428aec0e7168dcdfd7be26c7f2909f5f4fc1885248216c11db1d2e770482419085c31ed6b41b5ef801ebc15d7759188b0e9124ca66591f0fd8a9618681fc607832d7958c8f738c89843f06e1909fc57c4005e211583816f4d013705d36306bd6c62adeeaed0f17071ca831a14b9ea8f5880db5a34364aa6763f308649f819f759f5d37eff3f0325aa68e9b450b765a6940adaf694afafab657ca9606871d93477e024f064ab81c408a922a2dd868cbe6dea767d111572c2fe93'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 504652653 bytes downloaded
Downloaded and uncompressed: fracture-multi-region-x-ray-data
Data source import complete.


---

# **Bone Fracture Detection using Neural Networks**

## **Introduction**

Bone fractures are a common medical issue requiring prompt detection. Machine learning, particularly neural networks, offers potential improvements in fracture detection from X-ray images. This assignment explores various neural network models to enhance diagnostic accuracy.

## **Problem Statement**

The task is to develop and evaluate several neural network models for detecting bone fractures from X-ray images. The objective is to determine which model provides the best performance for accurate fracture detection.

## **Objectives**

1. **Implement Neural Network Architectures and Deep Learning Models**
2. **Evaluate and Compare Various Models**
3. **Visualize and Analyze Results**
4. **Document Findings**

---

## **1. Import Libraries**


In [57]:
# Importing Dependencies
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Conv2D, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## **2. Load and Pre-process Images""**

In [58]:
# Defining a function to do necessary preprocessing
def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        try:
            img = Image.open(os.path.join(directory, filename))
            img = img.resize((128, 128))
            img = img.convert('RGB')
            img = np.array(img) / 255.0
            images.append(img)
        except OSError as e:
            print(f"Error loading {os.path.join(directory, filename)}: {e}")
            continue
    return images

In [61]:
# Defining a function to do necessary preprocessing
def load_and_preprocess_images(directory):
    images = []
    for filename in os.listdir(directory):
        try:
            img = Image.open(os.path.join(directory, filename))
            img = img.resize((128, 128))
            img = img.convert('RGB')
            img = np.array(img) / 255.0
            images.append(img)
        except OSError as e:
            print(f"Error loading {os.path.join(directory, filename)}: {e}")
            continue
    return np.array(images)

In [62]:
# Loading Data
fractured_test = load_and_preprocess_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test/fractured')
fractured_train = load_and_preprocess_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/fractured')
fractured_val = load_and_preprocess_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val/fractured')
not_fractured_test = load_and_preprocess_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test/not fractured')
not_fractured_train = load_and_preprocess_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured')
not_fractured_val = load_and_preprocess_images('/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val/not fractured')


Error loading /kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test/not fractured/IMG0004347.jpg: image file is truncated (40 bytes not processed)
Error loading /kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test/not fractured/IMG0004148.jpg: image file is truncated (14 bytes not processed)
Error loading /kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test/not fractured/IMG0004149.jpg: image file is truncated (33 bytes not processed)
Error loading /kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test/not fractured/IMG0004308.jpg: image file is truncated (40 bytes not processed)
Error loading /kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification

In [63]:
# Creating labels
fractured_train_labels = np.ones(len(fractured_train))
fractured_val_labels = np.ones(len(fractured_val))
fractured_test_labels = np.ones(len(fractured_test))

not_fractured_train_labels = np.zeros(len(not_fractured_train))
not_fractured_val_labels = np.zeros(len(not_fractured_val))
not_fractured_test_labels = np.zeros(len(not_fractured_test))

In [ ]:
# Combining fractured and non-fractured data
X_train = np.concatenate([fractured_train, not_fractured_train], axis=0)
y_train = np.concatenate([fractured_train_labels, not_fractured_train_labels], axis=0)

X_val = np.concatenate([fractured_val, not_fractured_val], axis=0)
y_val = np.concatenate([fractured_val_labels, not_fractured_val_labels], axis=0)

X_test = np.concatenate([fractured_test, not_fractured_test], axis=0)
y_test = np.concatenate([fractured_test_labels, not_fractured_test_labels], axis=0)


In [ ]:
# Visualizing the data
fig, axes = plt.subplots(1, 6, figsize=(20, 10))
axes[0].imshow(fractured_test[0])
axes[1].imshow(fractured_train[0])
axes[2].imshow(fractured_val[0])
axes[3].imshow(not_fractured_test[0])
axes[4].imshow(not_fractured_train[0])
axes[5].imshow(not_fractured_val[0])
axes[0].set_title('Fractured')
axes[1].set_title('Fractured')
axes[2].set_title('Fractured')
axes[3].set_title('Not Fractured')
axes[4].set_title('Not Fractured')
axes[5].set_title('Not Fractured')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()


## **3. Data Augmentation**

In [ ]:
# Setting up the ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

datagen.fit(X_train)

## **4. Defining Neural Network Architectures**
### **4.1 Custom CNN Model**

In [ ]:
# Custom CNN Model building

def create_custom_cnn(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

### **4.2 VGG16 Model**

In [ ]:
# VGG16 Model Building

def create_vgg16_model(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

### **4.3 ResNet50 Model**

In [ ]:
# ResNet50 Model building

def create_resnet50_model(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

### **4.4 InceptionV3 Model**

In [ ]:
# InceptionV3 Model

def create_inceptionv3_model(input_shape):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

### **5. Model Training**

In [ ]:
# Model Training
def train_model(model, X_train, y_train, X_val, y_val, batch_size=32, epochs=10):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                        validation_data=(X_val, y_val),
                        epochs=epochs)
    return history

In [ ]:
# Custom CNN Training
custom_cnn_model = create_custom_cnn((128, 128, 3))
history_custom_cnn = train_model(custom_cnn_model, X_train, y_train, X_val, y_val)

# VGG16 Training
vgg16_model = create_vgg16_model((128, 128, 3))
history_vgg16 = train_model(vgg16_model, X_train, y_train, X_val, y_val)

# ResNet50 Training
resnet50_model = create_resnet50_model((128, 128, 3))
history_resnet50 = train_model(resnet50_model, X_train, y_train, X_val, y_val)

# InceptionV3 Training
inceptionv3_model = create_inceptionv3_model((128, 128, 3))
history_inceptionv3 = train_model(inceptionv3_model, X_train, y_train, X_val, y_val)


##**6. Model Evaluations**

In [ ]:
# **5. Evaluating the Models**

def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, cmap='Blues')
    plt.title('Confusion Matrix')
    plt.colorbar()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
# Custom CNN Evaluation
print("Custom CNN Evaluation:")
evaluate_model(custom_cnn_model, X_test, y_test)

# VGG16 Evaluation
print("VGG16 Evaluation:")
evaluate_model(vgg16_model, X_test, y_test)

# ResNet50 Evaluation
print("ResNet50 Evaluation:")
evaluate_model(resnet50_model, X_test, y_test)

# InceptionV3 Evaluation
print("InceptionV3 Evaluation:")
evaluate_model(inceptionv3_model, X_test, y_test)

## **7. Visualizing Training Results**

In [ ]:
# Visualizing Training Results

def plot_training_history(history, model_name):
    plt.figure(figsize=(12, 4))

    # Plotting training & validation accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plotting training & validation loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [ ]:
# Custom CNN Training History
plot_training_history(history_custom_cnn, "Custom CNN")

# VGG16 Training History
plot_training_history(history_vgg16, "VGG16")

# ResNet50 Training History
plot_training_history(history_resnet50, "ResNet50")

# InceptionV3 Training History**
plot_training_history(history_inceptionv3, "InceptionV3")